In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import os
import functools
from collections import defaultdict
import hashlib

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

_g = globals()

**Chunk Data**
---
Chunk through the charts data and create bins that are hash maps of the hadm_id key.

In [4]:
datadir = 'data/'
table_name = 'chartevents'
fname = table_name.upper() + '.csv.gz'

data = pd.read_csv(f'data/{fname}', dtype=str, encoding='latin1', 
                   compression='gzip', chunksize=(10**7)*2)

def append_to_csv(data, fname):
    if os.path.exists(fname):
        data.to_csv(fname, index=False, mode='a')
    else:
        data.to_csv(fname, index=False)

for i, chunk in enumerate(data):
    print(f'chunk {i} --> {chunk.shape}')
    chunk.columns = [x.lower() for x in chunk]
    chunk.hadm_id = [hashlib.md5(x.encode()).hexdigest()[0] for x in chunk.hadm_id.values]
    for bin_ in chunk.hadm_id.unique():
        sub = chunk.loc[chunk.hadm_id == bin_]
        append_to_csv(sub, 'split-data/' + table_name + '/' + f'bin_{bin_}.csv')

chunk 14 --> (20000000, 15)
chunk 15 --> (20000000, 15)
chunk 16 --> (10712483, 15)
